In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_data
import c3d_model
import math
import numpy as np
from tools import placeholder_inputs
from tools import _variable_with_weight_decay
from tools import tower_loss
from tools import tower_acc
from tools import _variable_on_cpu
from tools import _variable_with_weight_decay
from tools import average_gradients
from tools import get_logits
import input_data_for_extract_feature as input_data
# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1 , 'Batch size.')
FLAGS = flags.FLAGS
model_name = "../sports1m_finetuning_ucf101.model"
test_list_file = 'list/test.list'
unit_of_clip=2

    
def run_test():
    num_test_videos = len(list(open(test_list_file,'r')))
    print("Number of test videos={}".format(num_test_videos))

    images_placeholder, labels_placeholder = placeholder_inputs(FLAGS.batch_size * gpu_num)
    feature_placeholder, _ = placeholder_inputs(FLAGS.batch_size * gpu_num*unit_of_clip)

    fc6 = get_logits(images_placeholder, labels_placeholder,FLAGS,gpu_num)     
    
    saver = tf.train.Saver()
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    init = tf.global_variables_initializer()######
    sess.run(init)
    saver.restore(sess, model_name)
    
    next_batch_start = 0
    all_steps = int((num_test_videos - 1) / (FLAGS.batch_size * gpu_num) + 1)
    
    
    writer = tf.python_io.TFRecordWriter('fc6.tfrecord')
    fc6_batch_size=FLAGS.batch_size
    for step in xrange(all_steps):
#         try:
        np_arr_data, np_arr_label, next_batch_start, _, _ = input_data.read_clip_and_label(
                    test_list_file,FLAGS.batch_size,unit_of_clip,start_pos=next_batch_start)     
        fc6_of_batch=[]
        print("np_arr_data",np.shape(np_arr_data))
        batch_seq_arr = np_arr_data.reshape((fc6_batch_size * unit_of_clip, 16,112,112,3))
        print("batch_seq_arr",np.shape(batch_seq_arr))

        for batch_unit_index in range(0,unit_of_clip):
            temp_batch_data = batch_seq_arr[batch_unit_index*fc6_batch_size:batch_unit_index*fc6_batch_size + fc6_batch_size]
            batch_seq_fc6 = sess.run(fc6,feed_dict={images_placeholder:temp_batch_data})
            fc6_of_batch.append(batch_seq_fc6)
            #print("np.shape(batch_seq_fc6)",np.shape(batch_seq_fc6))
        print("np.shape(fc6_of_batch)",np.shape(fc6_of_batch))
        fc6_of_batch = np.array(fc6_of_batch)
        fc6_feature_batch = fc6_of_batch.reshape((fc6_batch_size,unit_of_clip * 4096))
        print("np.shape(fc6_feature_batch)",np.shape(fc6_feature_batch))
        print("np_arr_label",np_arr_label)
        #print(fc6_feature_batch)

        for batch_index in range(fc6_batch_size):            
            try:
                #image = io.imread(images[i]) # type(image) must be array!
                data = fc6_feature_batch[batch_index]
                data_raw = data.tostring()
                #label = int(labels[i])
                label = [np_arr_label[batch_index]]
                example = tf.train.Example(features=tf.train.Features(feature={
                                'label':tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
                                'image_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[data_raw]))}))
                writer.write(example.SerializeToString())
            except IOError as e:
                print('Could not read:', images[i])
                print('error: %s' %e)
                print('Skip it!\n')       
        writer.close()
#         except:
#             print("Error!")
    print("Done!")

def main(_):
    run_test()

if __name__ == '__main__':
    tf.app.run()

Number of test videos=538
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/141...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/142...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/143...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/144...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/145...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/146...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/147...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/148...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/149...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_01/150...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [0 0 0 0 0 0 0 0 0 0]
Loading a video clip

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/170...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/171...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/172...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [1 1 1 1 1 1 1 1 1 1]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/173...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/174...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/175...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/176...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/177...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/178...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_02/179...
Loading a video clip from /samba/HDD/dataset_e

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/195...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/196...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/197...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/198...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/199...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_03/200...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/141...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/142...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [2 2 2 2 2 2 2 2 3 3]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/143...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_04/144...
Loading a video clip from /samba/HDD/dataset_e

np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [4 4 4 4 4 4 4 4 4 4]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/163...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/164...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/165...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/166...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/167...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/168...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/169...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/170...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/171...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_05/172...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/188...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/189...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/190...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/191...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/192...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [5 5 5 5 5 5 5 5 5 5]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/193...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/194...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/195...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/196...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_06/197...
Loading a video clip from /samba/HDD/dataset_e

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/154...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/155...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/156...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/157...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/158...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/159...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/160...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/161...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/162...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [7 7 7 7 7 7 7 7 7 7]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_08/163...
Loading a video clip from /samba/HDD/dataset_e

Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/181...
np_arr_data (10, 2, 16, 112, 112, 3)
batch_seq_arr (20, 16, 112, 112, 3)
np.shape(fc6_of_batch) (2, 10, 4096)
np.shape(fc6_feature_batch) (10, 8192)
np_arr_label [8 8 8 8 8 8 8 8 8 8]
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/182...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/183...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/184...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/185...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/186...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/187...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/188...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/189...
Loading a video clip from /samba/HDD/dataset_elec_oneplustwo/class_09/190...
Loading a video clip from /samba/HDD/dataset_e

ValueError: Cannot feed value of shape (8, 16, 112, 112, 3) for Tensor 'Placeholder:0', which has shape '(10, 16, 112, 112, 3)'

In [1]:
%tb

No traceback available to show.
